# Jeans Equation Stuff

<img src="../images/jeans_eq.png" alt="jeans equation" style="height: 50px;">
<img src="../images/jeans_key.png" alt="jeans equation" style="height: 200px;">

In [14]:
from stellarutil import Simulation
from IPython.display import clear_output
import numpy as np

sim = Simulation(simulation_name="m10v_res250md", species=['star', 'dark', 'gas'])
clear_output()
halo = sim.get_halo()

# The Predicted Mass - 3 x G^(-1) x <σ^2> x r_half

In [19]:
# Find r_half (radius where half of the all star mass is)...

# Sort stars by 3D radius, stars closer to the center first
sorted_stars = sorted(halo.stars, key=lambda star: star.get_3DR())
# Get the total mass of all stars
TOTAL_STAR_MASS = np.sum([star.m for star in sorted_stars])
print(f"Total Star Mass: {TOTAL_STAR_MASS}")
# Go through stars until half of the mass is enclosed
enclosed_mass, star_index = 0, 0
while True:
    enclosed_mass += sorted_stars[star_index].m
    star_index += 1
    if enclosed_mass >= TOTAL_STAR_MASS / 2: break

# The radius of the star at star_index is the half mass radius
r_half = sorted_stars[star_index].get_3DR()
print(f"r_half: {r_half} at star {star_index} with mass {enclosed_mass}")

included_stars = sorted_stars[:star_index]

# Find <σ^2>...

los_dispersion = np.std([star.vz for star in included_stars])
luminosities = [star.m for star in included_stars]
average = np.average([los_dispersion ** 2] * len(included_stars), weights=luminosities)

# Plug and chug...

G =  4.3 * 10**-6 
prediction = 3 * pow(G, -1) * average * r_half
print()
print(f"Predicted Total Mass: {prediction:.2e} M☉")


Total Star Mass: 154166.9375
r_half: 4.783366634644828 at star 391 with mass 77164.56632232666

Predicted Total Mass: 1.17e+08 M☉


# The Actual Mass - M_(1/2)

In [20]:
# Find M_half (actual mass of every star, dark matter, and gas particle within r_half)

# Get the star masses using stellar util
star_masses = enclosed_mass
print(f"Star Masses by Hand:\t\t{ np.sum(star_masses):.2e} M☉")
# Get the dm masses
dark_x = sim.particles['dark']['position'][:,0] - halo.xc
dark_y = sim.particles['dark']['position'][:,1] - halo.yc
dark_z = sim.particles['dark']['position'][:,2] - halo.zc
dark_m = sim.particles['dark']['mass']
dark_distances = np.sqrt(np.square(dark_x) + np.square(dark_y) + np.square(dark_z))
dark_masses = dark_m[dark_distances < r_half]
print(f"DM Masses by Hand:\t\t{ np.sum(dark_masses):.2e} M☉")
# Get the gas masses
gas_x = sim.particles['gas']['position'][:,0] - halo.xc
gas_y = sim.particles['gas']['position'][:,1] - halo.yc
gas_z = sim.particles['gas']['position'][:,2] - halo.zc
gas_m = sim.particles['gas']['mass']
gas_distances = np.sqrt(np.square(gas_x) + np.square(gas_y) + np.square(gas_z))
gas_masses = gas_m[gas_distances < r_half]
print(f"Gas Masses by Hand:\t\t{ np.sum(gas_masses):.2e} M☉")

actual_mass = np.sum(star_masses) + np.sum(dark_masses) + np.sum(gas_masses)
print(f"\nActual Total Mass by Hand:\t{actual_mass:.2e} M☉")

Star Masses by Hand:		7.72e+04 M☉
DM Masses by Hand:		4.44e+08 M☉
Gas Masses by Hand:		1.22e+07 M☉

Actual Total Mass by Hand:	4.57e+08 M☉
